In [1]:
import nptdms as tdms
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import time

In [2]:
IS_TRANSFER = False
IS_MERGE    = False
IS_CUT      = False


# transfer TDMS to CSV

In [3]:
if IS_TRANSFER:
# voltage 
    dir_file = "./电流电压/" 
    filename = "Voltage-Current-2019-11-19T08-55-08"
    suffix = ".tdms"
    dir_save = "./voltage_current_csv/"
    group_name = 'Voltage-Current'


    for (_,_,files) in os.walk(dir_file):
        df_file = pd.DataFrame()
        df_file['file'] = files
        df_file.to_csv("./temp/voltage_files.txt",index = False)
        for file in files:
            if file[-5:]==".tdms":
                filename = file[:-5]
                file_read = dir_file+filename+suffix
                file_save = dir_save+filename+".csv"
                try:
                    # read file
                    tdms_file = tdms.TdmsFile(file_read)
                    # transform tdms into dataframe
                    df = tdms_file.as_dataframe()
                    # change the columns name
                    df.columns = ["time","Ic","U_IGBT3_1","U_IGBT3_2","Uce"]

                    # drop some nansense columns
        #             df.drop(["N0","N1"],axis = 1,inplace = True)
                    try:
                        df.dropna(axis = 0,inplace = True)
                    except:
                        pass
                    # transform the string time into timestamp
                    timestamp = []
                    for t in df.time:
                        if t != "":
                            timestamp.append(time.mktime(time.strptime(t[0:-7],"%Y/%m/%dT%H:%M:%S"))+ float(t[-7:]))
                        else:
                            timestamp.append(0)
                    df["timestamp" ]= timestamp
                    df["timestamp"].replace(0,np.NaN,inplace = True)
                    # save file
                    df.to_csv(file_save)
                except:
                    print(file)
                    pass
      

In [4]:
if IS_TRANSFER:
    # temperature
    dir_file = "./温度/" 
    filename = "Temperature-2019-11-19T08-55-08"
    suffix = ".tdms"
    dir_save = "./temperature_csv/"


    for (_,_,files) in os.walk(dir_file):
        df_file = pd.DataFrame()
        df_file['file'] = files
        df_file.to_csv("./temp/temperature_files.txt",index = False)
        for file in files:
            if file[-5:]==".tdms":
                filename = file[:-5]
                file_read = dir_file+filename+suffix
                file_save = dir_save+filename+".csv"

                try:
                    # read file
                    tdms_file = tdms.TdmsFile(file_read)
                    # transform tdms into dataframe
                    df = tdms_file.as_dataframe()
                    # change the columns name
                    try:
                        df.columns = ["time","Tbase_Cu","Tsink_IGBT3_2","Tsink_IGBT3_1","Tj_IGBT3_2","Tj_IGBT3_1"]
                    except:
                        df.columns = ["time","Tbase_Cu","Tsink_IGBT3_2","Tsink_IGBT3_1","Tj_IGBT3_2","Tj_IGBT3_1",
                                      "Tsink_IGBT4_2","Tsink_IGBT4_1","Tj_IGBT4_1"]
                    # drop some nansense columns
                    try:
                        df.dropna(axis = 0,inplace = True)
                    except:
                        pass
                    # transform the string time into timestamp
                    timestamp = []
                    for t in df.time:
                        if t != "":
                            timestamp.append(time.mktime(time.strptime(t[0:-7],"%Y/%m/%dT%H:%M:%S"))+ float(t[-7:]))
                        else:
                            timestamp.append(0)
                    df["timestamp" ]= timestamp
                    df["timestamp"].replace(0,np.NaN,inplace = True)
                    # save file
                    df.to_csv(file_save)

                except:
                    print(file)
                    pass

## merge temperature and voltage data on time 

In [5]:
if IS_MERGE:
    dir_temp = "./temperature_csv/" 
    dir_volt = "./voltage_current_csv/" 
    prefix_temp = "Temperature-"
    prefix_volt = "Voltage-Current-"
    time_str = "2019-12-09T17-58-55"
    suffix = ".csv"
    dir_save = "./temperature_voltage_current_csv/"

    file_err = []
    for (_,_,files) in os.walk(dir_volt):
        for file in files:
            time_str = file[-23:-4]
            # read file
            try:
                file_temp = dir_temp+prefix_temp+time_str+suffix
                file_volt = dir_volt+prefix_volt+time_str+suffix
                volt_df = pd.read_csv(file_volt,index_col = 0)
                temp_df = pd.read_csv(file_temp,index_col = 0)
            except:
                print("read error: ",time_str)
                file_err.append(file)

            try:
                # 去除不规整的volt数据
                select_df = volt_df.dropna(axis = 0)
                l = select_df.index.values
                m= np.where(l[1:]-l[:-1]==51)
                volt_df = volt_df.loc[l[m[0][0]]:,:]
            except:
                print("select error: ",time_str)
                file_err.append(file)

            try:
                # 重新定义时间
                volt_df["t"] = volt_df["timestamp"].round(1)
                volt_df.drop(["timestamp","time"],axis = 1,inplace = True )
                temp_df["t"] = temp_df["timestamp"].round(1)
                temp_df.drop(["timestamp","time"],axis = 1,inplace = True )
            except:
                print("def t error: ",time_str)
                file_err.append(file)

            try:
                # merge temp to voltage
                merge_df = pd.merge(volt_df,temp_df,on = "t", how = "left")
                # save
                merge_df.to_csv(file_volt,index = False)
                # merge voltage to temperature
                merge_df = pd.merge(temp_df,volt_df,on = "t", how = "left")
                # save
                merge_df.to_csv(file_temp,index = False)
            except:
                print("merge error: ",time_str)
                file_err.append(file)

    err_file_df = pd.DataFrame()
    err_file_df["file"] = file_err
    err_file_df.to_csv("./temp/error_files.txt",index = False)

# cut into cycle

In [6]:
if IS_CUT:
    # cut voltage file into cycles
    dir_volt = "./voltage_current_csv/" 
    prefix_volt = "Voltage-Current-"
    time_str = "2019-12-09T17-58-55"
    suffix = ".csv"

    n_round = 0
    for (_,_,files) in os.walk(dir_volt):
        for file in files:
            time_str = file[-23:-4]
            try:
                # read file
                file_volt = dir_volt+prefix_volt+time_str+suffix
                volt_df = pd.read_csv(file_volt)
                # get index of max T and min T
                select_df = volt_df.dropna(inplace = False)
                l = select_df["Tj_IGBT3_1"].values
                l[1:-1] = (l[:-2]+l[1:-1]+l[2:])/3
                ix = select_df.index.values  # index of temp
                m = l[1:]-l[:-1]> 0
                m = m.astype(np.int8)
                n = m[1:]-m[:-1]
                n = np.insert(n,0,values = 1)
                n = np.append(n,values = 1)  # 1 0 0 0 -1 0 0 1 0 0 -1 0 0 1 //the same size as ix
                i_Tmax = np.where(n==-1)[0]  # 
                i_Tmin = np.where(n==1)[0]

                # cut and save
                for i in range(len(i_Tmax)):
                    n_round += 1
                    dur = i_Tmin[i+1]-i_Tmin[i]
                    Tmax = l[i_Tmax[i]]
                    Tmin1 = l[i_Tmin[i]]
                    Tmin2 = l[i_Tmin[i+1]]
                #     print(n_round,dur,Tmax,Tmin1,Tmin2)
                    df = volt_df.loc[ix[i_Tmin[i]+1]:ix[i_Tmin[i+1]],:]
                    df.to_csv("./voltage_current_cycle/"+str(n_round)+"-"+str(dur)+"-"+str(int(Tmin1))+"-"+str(int(Tmax))+"-"+str(int(Tmin2))+".csv",index = False)
            except:
                print(time_str)



In [7]:
if IS_CUT:
    dir_temp = "./temperature_csv/" 
    prefix_temp = "Temperature-"
    # time_str = "2019-12-09T17-58-55"
    suffix = ".csv"

    n_round = 0
    for (_,_,files) in os.walk(dir_volt):
        for file in files:
            time_str = file[-23:-4]
            try:
                # read file
                file_volt = dir_temp+prefix_temp+time_str+suffix
                volt_df = pd.read_csv(file_volt)
                # get index of max T and min T
    #             select_df = volt_df.dropna(inplace = False)
                l = volt_df["Tj_IGBT3_1"].values
                l[1:-1] = (l[:-2]+l[1:-1]+l[2:])/3
                ix = volt_df.index.values
                m = l[1:]-l[:-1]> 0
                m = m.astype(np.int8)
                n = m[1:]-m[:-1]
                n = np.insert(n,0,values = 1)
                n = np.append(n,values = 1)
                i_Tmax = np.where(n==-1)[0]
                i_Tmin = np.where(n==1)[0]

                # cut and save
                for i in range(len(i_Tmax)):
                    n_round += 1
                    dur = (i_Tmin[i+1]-i_Tmin[i])/10
                    Tmax = l[i_Tmax[i]]
                    Tmin1 = l[i_Tmin[i]]
                    Tmin2 = l[i_Tmin[i+1]]
                #     print(n_round,dur,Tmax,Tmin1,Tmin2)
                    df = volt_df.loc[ix[i_Tmin[i]+1]:ix[i_Tmin[i+1]],:]
                    df.to_csv("./temperature_cycle/"+str(n_round)+"-"+str(int(dur))+"-"+str(int(Tmin1))+"-"+str(int(Tmax))+"-"+str(int(Tmin2))+".csv",index = False)
            except:
                print(time_str)



# collect Temperature info to choose useful csv

In [2]:
# collect all temperature data
dir_temp = "./temperature_cycle/" 
suffix = ".csv"

all_df = pd.DataFrame(columns = ["round","dur_second","T_min1","T_max","T_min2"])
for (_,_,files) in os.walk(dir_temp):
    for file in files:
        try:
            l = file[:-4].split("-")
            l = [i for i in l if i!=""]
            l = np.array([int(i) for i in l]).reshape(1,5)
            df = pd.DataFrame(l, columns = ["round","dur_second","T_min1","T_max","T_min2"])
            all_df = pd.concat([all_df,df],axis = 0)
        except:
            print(file)

all_df.sort_values(by = 'round',inplace = True)
all_df.to_csv("./temp/all_temp_info.csv",index = False)    
print(all_df.shape)
all_df.head()    

(86944, 5)


,round,dur_second,T_min1,T_max,T_min2
0,1,10,19,41,35
0,2,20,35,62,41
0,3,20,41,73,49
0,4,20,49,89,58
0,5,20,58,99,65


In [3]:
# choose useful cycles
# read csv
all_df  = pd.read_csv("./temp/all_temp_info.csv")
print("raw shape:",all_df.shape)

# culculate delta
all_df['T_delta1'] = all_df['T_max']-all_df['T_min1']
all_df['T_delta2'] = all_df['T_max']-all_df['T_min2']

# delete some error data
all_df = all_df.loc[all_df['T_max'].values<180,:]
all_df = all_df.loc[all_df['T_min1'].values>10,:]
all_df = all_df.loc[all_df['T_min2'].values>10,:]
print("shape after delete error:",all_df.shape)

# select the useful data
# Tmax [160,175]
# Tmin1 [80,100]
# Tmin2 [80,100]
# Tdelta1 [65,95]
# Tdelta2 [65,95]
# during time > 25 second
s = np.all( [all_df['T_max'].values > 168, all_df['T_max'].values <178,
             all_df['T_min1'].values > 83, all_df['T_min1'].values < 92,
             all_df['T_min2'].values > 83, all_df['T_min2'].values < 92,
#              all_df['T_delta1'].values > 65, all_df['T_delta1'].values <95,
            ],
           axis = 0)
all_df = all_df.loc[ s,:]
all_df.to_csv("./temp/all_useful_df.csv")
print("shape of useful data:",all_df.shape)
all_df.head()

raw shape: (86944, 5)
shape after delete error: (86866, 7)
shape of useful data: (67209, 7)


,round,dur_second,T_min1,T_max,T_min2,T_delta1,T_delta2
5336,5337,28,88,170,88,82,82
5337,5338,28,88,170,88,82,82
5338,5339,28,88,170,88,82,82
5339,5340,28,88,170,88,82,82
5340,5341,28,88,170,88,82,82


In [ ]:
# choose IGBT3_1 data only
all_df = pd.read_csv("./temp/all_useful_df.csv")

save_temp = "./temperature_IGBT3_1/" 
dir_temp  = "./temperature_cycle/" 
suffix = ".csv"
str_info = "15-40-86-165-94"

for i in range(all_df.shape[0]):
    str_info = str(all_df['round'].values[i])+ "-"+ str(all_df['dur_second'].values[i])+ "-" +str(all_df['T_min1'].values[i])+ "-" +str(all_df['T_max'].values[i])+ "-"+ str(all_df['T_min2'].values[i])
    filename = dir_temp+ str_info +suffix
    try:
        temp_df  = pd.read_csv(filename)
        temp_df = temp_df.loc[:,["Tbase_Cu","Tsink_IGBT3_1","Tj_IGBT3_1","t","Ic","Uce","Power","dT_IGBT3_1","Rth_IGBT3_1"]]
        temp_df.to_csv(save_temp+ str_info +suffix, index = False)
    except:
        print(filename)
        